In [ ]:
# !pip install imbalanced_learn --quiet
# After installation restart the kernel as it will complaint 

In [ ]:
from utils.shuqing_all import *
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline
plt.style.use('fivethirtyeight')
five_thirty_eight=[
   "#30a2da",
    "#fc4f30",
    "#e5ae38",
    "#6d904f",
    "#8b8b8b",      
]
import os
!pip install haversine -qq
from haversine import haversine
from sklearn.preprocessing import LabelEncoder
pd.set_option('display.max_columns', 500)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score

In [ ]:
PROJECT_ID='uw-data-warehouse-prod'

In [ ]:
### ORIGINAL DATA COMING FROM https://www.freemaptools.com/download/outcode-postcodes/postcode-outcodes.csv

outcodes = pd.read_csv('postcode-outcodes.csv').loc[:, 'postcode':]
outcodes = outcodes.loc[outcodes.latitude != 0].reset_index(drop=True)

In [ ]:
latlongs=outcodes.set_index('postcode').values

In [ ]:
%%time
distances =[[haversine(p,q) for q in latlongs] for p in latlongs]

In [ ]:
distance_df=pd.DataFrame(distances,index=outcodes.postcode.tolist(),columns=outcodes.postcode.tolist())
unstacked_dis=distance_df.unstack().reset_index()
unstacked_dis.columns=['from', 'to', 'distance_km']

In [ ]:
p_cohort = f"""SELECT partner_position_id 
FROM `uw-partner-prod.metrics_analysis.partner_position_master_data_current`
WHERE is_live AND customers_gathered_last_1y > 0
"""
partner_postcodes_q = """SELECT partner_position_id
        ,UPPER(partner.address.post_code) AS postcode, 
  FROM `uw-partner-prod.position_v1.partner_position_ownership_changed_latest` owner
  LEFT JOIN `uw-partner-prod.partner_v1.partner_details_changed_latest` details
  ON owner.owner_id = details.partner_id
  ,UNNEST (partner.address.lines) AS address_lines
  GROUP BY 1, 2"""
bb_q="""
SELECT * FROM `uw-data-models-prod.marketing_dataform_models.eero_source`
"""
marketing_pre="""
SELECT
customer.custaccountno  AS customer_cust_account_number,
(
CASE WHEN customer.custnomarketing is null and customer.custenddate is null  THEN 'Yes' 
ELSE 'No' 
END
) AS customer_cust_is_marketing_subscribed
FROM  `uw-data-models-prod.marketing_dataform_models.customer_model` AS customer
GROUP BY
    1,
    2
ORDER BY
    1  
"""
rfm_score="""
SELECT * FROM `uw-data-models-prod.partner_dataform_models_position.rfm_score`"""

title_df="""

SELECT * FROM `uw-data-models-prod.partner_dataform_models_position.position` """



In [ ]:
bb=download_data(project_id=PROJECT_ID,query=bb_q) 

In [ ]:
partner_cohort=download_data(project_id=PROJECT_ID,query=p_cohort) 

In [ ]:
partner_postcodes=download_data(project_id=PROJECT_ID,query=partner_postcodes_q)

In [ ]:
bq_rfm=download_data(project_id=PROJECT_ID,query=rfm_score)

In [ ]:
title_df=download_data(project_id=PROJECT_ID,query=title_df)

In [ ]:
group_1 = ['D']
group_2 = ['QD', 'TL', 'STL']
group_3 = ['GL', 'SGL', 'NNL', 'NGL']

group_map = {'D': 'group_1'}
for t in group_2: group_map[t] = 'group_2'
for t in group_3: group_map[t] = 'group_3'
title_df['title_group']=title_df.title.map(group_map).astype('category')
partners_title=title_df[['partner_position_id','title_group']].copy()
bq_rfm=bq_rfm.merge(partners_title,left_on=['partner_position_id'],right_on=['partner_position_id'],how='left')


In [ ]:
caci_q= """
SELECT 
custaccountno,
REGEXP_EXTRACT(UPPER(custpostcode), '^[^ ]*') as outcode,
age_1,

lukcat_agebandn,
lukp_a2_age1,
lukp_a2_age2,
lukp_a2_age3,
lukp_a2_age4,
lukp_a2_age5,
lukcat_incomeband2,
lukp_a2_income1,
lukp_a2_income2,
lukp_a2_income3,
lukp_a2_income4,
lukp_a2_income5,
lukp_a2_income6a,
lukp_a2_income6b,
lukp_a2_income6c,
lukp_a2_hhd1,
lukp_a2_hhd2,
lukp_a2_hhd5,
lukp_a2_hhd6,
lukcat_children,
lukcat_a2_childage,
lukcat_hhdsize,
lukcat_beds,
lukp_occd2,
lukp_occd3,
lukp_occd5,
lukp_occd6,
lukp_occd8,
lukp_occd9,
lukp_occd10,
lukp_occd11,
lukp_occd12,
lukp_a2_house5,
lukp_a2_house6,
lukp_a2_house7,
lukp_a2_house8,
lukp_a2_house1,
lukp_a2_house2,
lukp_a2_house3,
lukp_netfreq10p,
ips_intaddict,
ips_usetablet,
ips_useconsole,
lukp_mobhas,
ips_haspc,
oc14_bghtemailad,
oc14_willpaymore,
oc14_worthpaymore,
oc14_mobweb_wknd1,
oc14_mobweb_wknd2,
oc14_mobweb_wknd3,
ips_apps_games,
ips_apps_music,
ips_apps_transp,
ips_apps_date,
ips_apps_deliv,
ips_regaccessmobile2,
ips_regchat,
ips_tvfilmsonline,
ips_musicstreamfree,
ips_musicstreampaid,
ips_regpodcast,
ips_netflix,
ips_amazonprimetv,
oc14_vdemand_mob,
oc14_vdemand_pc,
oc14_vdemand_tv,
ips_vodwads,
oc14_buynewtech,
oc14_needmobnet,
oc14_cheapertech,
ips_acceptlbs,
ips_acceptvoa,
ips_followads,
ips_mobascheckout,
ips_mobentertain,
ips_mobimportant,
fr_smartband,
ips_smthomeappl,
ips_smthomeassist,
ips_smthomeutil,
ips_tvstreaming,
oc14_homesmartspeak,
oc14_homesmartlight,
oc14_homewifiplugs,
ips_wrlspeak,ips_mobmanyapps,
ips_mobreggames,ips_likesfactads,oc14_bghttvad,oc14_bghtnewspmagad,oc14_bghtinsertad,
oc14_bghtnetad,oc14_bghtmobilead,oc14_bghtleafletad,oc14_respdm



FROM `uw-eero-caci-data.oceans.oceans_customers`

"""

In [ ]:
caci=download_data(project_id=PROJECT_ID,query=caci_q)

In [ ]:
marketing=download_data(project_id=PROJECT_ID,query=marketing_pre)

In [ ]:
marketing['customer_cust_account_number']=marketing.customer_cust_account_number.astype('int')
marketing.drop_duplicates(subset='customer_cust_account_number',inplace=True)

In [ ]:
partner_postcodes['outcode']=partner_postcodes.copy().postcode.str.split().str[0]
partner_postcodes=partner_postcodes[partner_postcodes.partner_position_id.isin(
    partner_cohort.partner_position_id)].dropna().reset_index(drop=True)

In [ ]:
partner_cohort.shape,partner_postcodes.shape

In [ ]:
partner_outcodes=partner_postcodes.groupby(['outcode']).agg(list)[['partner_position_id']]

In [ ]:
le=LabelEncoder()
bb['has_eeros']=le.fit_transform(bb.has_eeros)
bb_no_duplicates=bb.sort_values(by='has_eeros',ascending=False).drop_duplicates(subset='customer_cust_account_number',keep='first').copy()

In [ ]:
bb_no_duplicates.has_eeros.value_counts()/bb_no_duplicates.shape[0],bb_no_duplicates.shape[0]

In [ ]:
bb_no_duplicates['customer_cust_account_number']=bb_no_duplicates.customer_cust_account_number.astype('int')

In [ ]:
df=bb_no_duplicates.merge(caci,left_on=['customer_cust_account_number'],right_on=['custaccountno'],how='left').drop(['broad_call_connections_cli_number','broad_call_connections_number_of_services','custaccountno'],axis=1)
df=df.merge(marketing,left_on='customer_cust_account_number',right_on='customer_cust_account_number',how='left')

In [ ]:
df.isnull().sum()  # --- 3% null values

In [ ]:
## Fillna
for c in ['broad_call_connections_equipment','broad_call_connections_broadband_tariff']:
    df[c].fillna(df[c].mode()[0],inplace=True)
df.outcode.fillna('Unknown',inplace=True)
for c in df.columns[5:-1].tolist():
    df[c].fillna(df[c].median(),inplace=True)

In [ ]:
df.isnull().sum().sum()

## EDA

In [ ]:
# Apply preferred target audience constraints
df.rename(columns={'age_1':'age'},inplace=True)
df['age_over_67']=np.where(df.age>67,True,False)
df.rename(columns={'lukp_netfreq10p':'Internet_frequent_user'},inplace=True)
# non_eeros customer base dropped from 255887 to 108984
eeros_df=df[df.has_eeros==1]
non_eeros=df[df.has_eeros==0]
non_eeros=non_eeros[(non_eeros.broad_call_connections_broadband_tariff=='Ultra')|(non_eeros.broad_call_connections_broadband_tariff=='Ultra+')]
non_eeros=non_eeros[non_eeros.customer_cust_is_marketing_subscribed=='Yes']
non_eeros=non_eeros[~non_eeros.age_over_67]
new_df=pd.concat([eeros_df,non_eeros])
new_df.drop(['customer_cust_is_marketing_subscribed'],axis=1,inplace=True)
new_df.reset_index(drop=True,inplace=True)

In [ ]:
# execute only for visualizationkeep df as new df for visualisation purposes
new_df=df.copy()

In [ ]:
eeros_df.shape[0]# 0.28%

In [ ]:
g=sns.countplot(x='age_over_67',data=df)
g.set_title('Age over 67 distribution',fontsize=19)
g.set_xlabel('over 67 ys',fontsize=17)
total=len(df)
for p in g.patches:
    height = p.get_height()
    g.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}%'.format(height/total*100),
            ha="center", fontsize=14)

In [ ]:
g=sns.countplot(x='has_eeros',data=new_df)
g.set_title('Eeros distribution',fontsize=19)
g.set_xlabel('has eeros',fontsize=17)
plt.yscale('log')
total=len(new_df)
for p in g.patches:
    height = p.get_height()
    g.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}%'.format(height/total*100),
            ha="center", fontsize=14)

In [ ]:
new_df.loc[(new_df.age>=18)&(new_df.age<25),'Age_band']='18-24'
new_df.loc[(new_df.age>=25)&(new_df.age<35),'Age_band']='24-34'
new_df.loc[(new_df.age>=35)&(new_df.age<45),'Age_band']='35-44'
new_df.loc[(new_df.age>=45)&(new_df.age<55),'Age_band']='45-54'
new_df.loc[(new_df.age>=55)&(new_df.age<=67),'Age_band']='55-67'
new_df.loc[(new_df.age>67),'Age_band']='67+'

In [ ]:
cols_to_drop=['age_over_67','age','lukcat_agebandn','lukp_a2_age1','lukp_a2_age2','lukp_a2_age3','lukp_a2_age4',
             'lukp_a2_age5','lukcat_incomeband2','lukp_a2_income1',
 'lukp_a2_income2',
 'lukp_a2_income3',
 'lukp_a2_income4',
 'lukp_a2_income5',
 'lukp_a2_income6a',
 'lukp_a2_income6b',
 'lukp_a2_income6c','lukp_a2_hhd1', 'lukp_a2_hhd2', 'lukp_a2_hhd5', 'lukp_a2_hhd6','lukp_occd2',
 'lukp_occd3',
 'lukp_occd5',
 'lukp_occd6',
 'lukp_occd8',
 'lukp_occd9',
 'lukp_occd10',
 'lukp_occd11',
 'lukp_occd12',
  'lukp_a2_house5',
 'lukp_a2_house6',
 'lukp_a2_house7',
 'lukp_a2_house8',
 'lukp_a2_house1',
 'lukp_a2_house2',
 'lukp_a2_house3' , 'ips_apps_games', 'ips_apps_music',
       'ips_apps_transp', 'ips_apps_date', 'ips_apps_deliv',
       'ips_regaccessmobile2', 'oc14_mobweb_wknd1', 'oc14_mobweb_wknd2', 'oc14_mobweb_wknd3','apps_music', 'apps_transp', 'apps_date', 'apps_deliv','regaccessmobile2',
'regchat', 'tvfilmsonline', 'musicstreamfree',
       'musicstreampaid', 'regpodcast', 'netflix', 'amazonprimetv',
       'vdemand_mob', 'vdemand_pc', 'vdemand_tv', 'vodwads', 'ips_regchat', 'ips_tvfilmsonline',
       'ips_musicstreamfree', 'ips_musicstreampaid', 'ips_regpodcast',
       'ips_netflix', 'ips_amazonprimetv', 'oc14_vdemand_mob',
       'oc14_vdemand_pc', 'oc14_vdemand_tv', 'ips_vodwads', 'oc14_buynewtech',
       'oc14_needmobnet', 'oc14_cheapertech',
       'ips_acceptvoa', 'ips_followads', 'ips_mobascheckout',
       'ips_mobentertain', 'ips_mobimportant',  'buynewtech',
       'needmobnet', 'cheapertech', 'acceptvoa', 'followads', 'mobascheckout',
       'mobentertain', 'mobimportant', 'ips_tvstreaming',  'tvstreaming'  , 'oc14_bghtemailad', 'oc14_willpaymore',
       'oc14_worthpaymore',  'ips_likesfactads','oc14_bghttvad',
       'oc14_bghtnewspmagad', 'oc14_bghtinsertad', 'oc14_bghtnetad',
       'oc14_bghtmobilead', 'oc14_bghtleafletad', 'oc14_respdm', 'oc14_homesmartspeak', 'oc14_homesmartlight',
       'oc14_homewifiplugs', 'ips_wrlspeak', 'ips_mobmanyapps',
       'ips_mobreggames', 'ips_likesfactads','oc14_bghttvad',
       'oc14_bghtnewspmagad', 'oc14_bghtinsertad', 'oc14_bghtnetad',
       'oc14_bghtmobilead', 'oc14_bghtleafletad', 'oc14_respdm' ,'ips_acceptlbs','acceptlbs','ips_usetablet', 'ips_useconsole', 'lukp_mobhas', 'ips_haspc',
       'fr_smartband', 'ips_smthomeappl', 'ips_smthomeassist',
       'ips_smthomeutil','usetablet', 'useconsole',
       'mobhas', 'haspc', 'bghtemailad', 'willpaymore', 'worthpaymore',
       'apps_games', 'smartband', 'smthomeappl', 'smthomeassist',
       'smthomeutil',  'homesmartspeak', 'homesmartlight',
       'homewifiplugs', 'wrlspeak', 'mobmanyapps', 'mobreggames',
       'likesfactads', 'bghttvad', 'bghtnewspmagad', 'bghtinsertad',
       'bghtnetad', 'bghtmobilead', 'bghtleafletad', 'respdm', ]

In [ ]:
new_df['income']=new_df['lukcat_incomeband2'].map({1.0: '0-9999', 
                                               2.0: '10000-19999', 
                                               3.0: '20000-29999',
                                                        4.0: '30000-39999',
                                                        5.0: '40000-49999',
                                                        6.0: '50000-74999',
                                                        7.0: '75000-99999',
                                                        8.0: '100000+'
                                               })

new_df['min_hh_size']=new_df[[f for f in new_df.columns if 'a2_hhd' in f]].idxmin(axis='columns')
hh_map={'lukp_a2_hhd1':'1 person', 'lukp_a2_hhd2':'2 persons', 'lukp_a2_hhd5':'3-4 persons', 'lukp_a2_hhd6':'5+ persons'}
new_df['min_hh_size']=new_df.min_hh_size.map(hh_map)
new_df['occupation']=new_df[[f for f in new_df.columns if 'occd' in f]].idxmax(axis='columns')
job_map={'lukp_occd2':'Director / Managerial',
 'lukp_occd3':'Looking after home',
 'lukp_occd5':'Office Worker',
 'lukp_occd6':'Professional',
 'lukp_occd8':'Retired',
 'lukp_occd9':'Is self employed',
 'lukp_occd10':'Service Sector',
 'lukp_occd11':'Skilled / Manual Worker',
 'lukp_occd12':'Student'}
new_df['occupation']=new_df.occupation.map(job_map)
new_df['ownership']=new_df[['lukp_a2_house1',
 'lukp_a2_house2',
 'lukp_a2_house3']].idxmax(axis='columns')
ownership_map={'lukp_a2_house1':'Owner-occupied',
 'lukp_a2_house2':'Social renting',
 'lukp_a2_house3':'Private renting'}
new_df['ownership']=new_df.ownership.map(ownership_map)
new_df['property_type']=new_df[[f for f in new_df.columns if 'a2_house' in f][:4]].idxmax(axis='columns')
property_map={'lukp_a2_house5':'Detached', 'lukp_a2_house6':'Flat/maisonette',
 'lukp_a2_house7':'Semi-detached', 'lukp_a2_house8':'Terraced '}
new_df['property_type']=new_df['property_type'].map(property_map)
new_df['wkn_internet_usage']=new_df[[f for f in new_df.columns if 'wknd' in f]].idxmax(axis='columns')
hour_map={'oc14_mobweb_wknd1':'< 1hr', 'oc14_mobweb_wknd2':'1-5hrs', 'oc14_mobweb_wknd3':'> 5hrs'}
new_df['wkn_internet_usage']=new_df.wkn_internet_usage.map(hour_map)

In [ ]:
for c in ['ips_acceptlbs', 'ips_usetablet', 'ips_useconsole', 'ips_tvstreaming',
       'lukp_mobhas', 'ips_haspc', 'oc14_bghtemailad', 'oc14_willpaymore',
       'oc14_worthpaymore',  'ips_apps_games', 'ips_apps_music',
       'ips_apps_transp', 'ips_apps_date', 'ips_apps_deliv',
       'ips_regaccessmobile2', 'ips_regchat', 'ips_tvfilmsonline',
       'ips_musicstreamfree', 'ips_musicstreampaid', 'ips_regpodcast',
       'ips_netflix', 'ips_amazonprimetv', 'oc14_vdemand_mob',
       'oc14_vdemand_pc', 'oc14_vdemand_tv', 'ips_vodwads', 'oc14_buynewtech',
       'oc14_needmobnet', 'oc14_cheapertech',
       'ips_acceptvoa', 'ips_followads', 'ips_mobascheckout',
       'ips_mobentertain', 'ips_mobimportant', 'fr_smartband',
       'ips_smthomeappl', 'ips_smthomeassist', 'ips_smthomeutil',
       'oc14_homesmartspeak', 'oc14_homesmartlight',
       'oc14_homewifiplugs', 'ips_wrlspeak', 'ips_mobmanyapps',
       'ips_mobreggames', 'ips_likesfactads','oc14_bghttvad',
       'oc14_bghtnewspmagad', 'oc14_bghtinsertad', 'oc14_bghtnetad',
       'oc14_bghtmobilead', 'oc14_bghtleafletad', 'oc14_respdm']:
    if len(c.split('_'))==2:    
        new_df[c.split('_')[-1]]=np.where(new_df[c]>0.5,True,False)
    else:
        new_df[c.split('_')[1]+'_'+c.split('_')[2]]=np.where(new_df[c]>0.5,True,False)
new_df['count_apps']=new_df[['apps_music', 'apps_transp', 'apps_date', 'apps_deliv','regaccessmobile2']].sum(axis=1)
new_df['count_streaming']=new_df[['regchat', 'tvfilmsonline', 'musicstreamfree',
       'musicstreampaid', 'regpodcast', 'netflix', 'amazonprimetv',
       'vdemand_mob', 'vdemand_pc', 'vdemand_tv', 'vodwads','tvstreaming']].sum(axis=1) 

new_df['count_tech_savvy']=new_df[['buynewtech',
       'needmobnet', 'cheapertech',
       'acceptvoa', 'followads', 'mobascheckout',
       'mobentertain', 'mobimportant']].sum(axis=1)
new_df['count_home_devices']=new_df[['usetablet', 'useconsole',
       'mobhas', 'haspc','smartband',
       'smthomeappl', 'smthomeassist', 'smthomeutil',
        'homesmartspeak', 'homesmartlight',
       'homewifiplugs', 'wrlspeak', 'mobmanyapps',
       'mobreggames']].sum(axis=1)

new_df['count_marketing_preference']=new_df[[ 'bghtemailad', 'willpaymore',
       'worthpaymore',  'likesfactads','bghttvad',
       'bghtnewspmagad', 'bghtinsertad', 'bghtnetad',
       'bghtmobilead', 'bghtleafletad', 'respdm','acceptlbs']].sum(axis=1)
     
          

In [ ]:
# preferred audience constraints applied
new_df.dropna(inplace=True)
new_df=new_df[new_df.count_home_devices>1]

In [ ]:
df.shape[0]-new_df.shape[0]

In [ ]:
new_df.shape

In [ ]:
model_df=new_df.drop(['age','age_over_67', 'min_hh_size', 'occupation', 'ownership', 'property_type',
       'acceptlbs', 'usetablet', 'useconsole', 'tvstreaming', 'mobhas',
       'haspc', 'bghtemailad', 'willpaymore', 'worthpaymore', 'apps_games',
       'apps_music', 'apps_transp', 'apps_date', 'apps_deliv',
       'regaccessmobile2', 'regchat', 'tvfilmsonline', 'musicstreamfree',
       'musicstreampaid', 'regpodcast', 'netflix', 'amazonprimetv',
       'vdemand_mob', 'vdemand_pc', 'vdemand_tv', 'vodwads', 'buynewtech',
       'needmobnet', 'cheapertech', 'acceptvoa', 'followads', 'mobascheckout',
       'mobentertain', 'mobimportant', 'smartband', 'smthomeappl',
       'smthomeassist', 'smthomeutil',  'homesmartspeak',
       'homesmartlight', 'homewifiplugs', 'wrlspeak', 'mobmanyapps',
       'mobreggames', 'likesfactads', 'bghttvad', 'bghtnewspmagad',
       'bghtinsertad', 'bghtnetad', 'bghtmobilead', 'bghtleafletad', 'respdm',
       'count_apps', 'count_streaming', 'count_tech_savvy',
       'count_home_devices', 'count_marketing_preference','wkn_internet_usage'],axis=1).copy()
visualization_df=new_df.drop(cols_to_drop,axis=1)


In [ ]:
visualization_df=new_df.drop(cols_to_drop,axis=1)

In [ ]:
def sort_cat_label(df):
    cat_list=['18-24','24-34','35-44','45-54','55-67','67+']
    cat_api=pd.api.types.CategoricalDtype(categories=cat_list,ordered=True)
    df['Age_band']=df['Age_band'].astype(cat_api)
    cat_list=['0-9999','10000-19999','20000-29999','30000-39999','40000-49999','50000-74999','75000-99999','100000+']
    cat_api=pd.api.types.CategoricalDtype(categories=cat_list,ordered=True)
    df.income=df.income.astype(cat_api)
    cat_list=['1 person','2 persons','5+ persons']
    cat_api=pd.api.types.CategoricalDtype(categories=cat_list,ordered=True)
    df.min_hh_size=df.min_hh_size.astype(cat_api)
    return df  


In [ ]:
visualization_df=sort_cat_label(visualization_df)

In [ ]:
for f in ['broad_call_connections_broadband_tariff', 
       'lukcat_children', 'lukcat_a2_childage', 'lukcat_hhdsize',
       'lukcat_beds',  'Age_band',
       'income', 'min_hh_size', 'occupation', 'ownership', 'property_type',
       'count_apps', 'count_streaming', 'count_tech_savvy',
       'count_home_devices', 'count_marketing_preference']:
    fig,ax=plt.subplots(figsize=(12,8))
    g=sns.countplot(x=f,data=visualization_df,hue='has_eeros',ax=ax)
    plt.yscale('log')
    g.set_title(f'{f} distribution',fontsize=19)
    g.set_ylabel('no. of eeros',fontsize=17)
    plt.xticks(rotation=45)
    total=len(visualization_df)
    for p in g.patches:
        height = p.get_height()
        g.text(p.get_x()+p.get_width()/2.,
                height + 3,
                '{:1.4f}%'.format(height/total*100),
                ha="center", fontsize=14)
    

In [ ]:
fig,ax=plt.subplots(figsize=(8,6))
visualization_df.groupby(['has_eeros'])[['ips_intaddict']].mean().plot(kind='bar',ax=ax,title='Interent Addiction Avg Score per Group')



# Cosine

In [ ]:
model_df['house_type'] = model_df[['lukp_a2_house5','lukp_a2_house6','lukp_a2_house7', 'lukp_a2_house8']].idxmax(axis=1)


model_df['house_type_revised'] = model_df['house_type'].map({'lukp_a2_house5': 'Detached', 
                                               'lukp_a2_house6': 'Flat_or_maisonnette', 
                                               'lukp_a2_house7': 'Semi_detached',
                                               'lukp_a2_house8': 'Terraced'})
model_df['house_usage'] = model_df[['lukp_a2_house1','lukp_a2_house2','lukp_a2_house3']].idxmax(axis=1)


model_df['house_usage_revised'] = model_df['house_usage'].map({'lukp_a2_house1': 'owner_occupied', 
                                               'lukp_a2_house2': 'social_renting', 
                                               'lukp_a2_house3': 'private_renting'
                                               })
# mapping income band
model_df['income_band'] = model_df['lukcat_incomeband2'].map({1.0: '0-9999', 
                                               2.0: '10000-19999', 
                                               3.0: '20000-29999',
                                                        4.0: '30000-39999',
                                                        5.0: '40000-49999',
                                                        6.0: '50000-74999',
                                                        7.0: '75000-99999',
                                                        8.0: '100000+'
                                               })
model_df[['lukcat_incomeband2', 'income_band']].head()

In [ ]:
model_df.columns=[col.lower() for col in model_df.columns]

In [ ]:
### remove all vars which contains repeated information
to_keep = ['customer_cust_account_number',
#  'broad_call_connections_equipment',
 'broad_call_connections_broadband_tariff',
#  'outcode',
 'age_band',

#  'lukcat_agebandn',
#  'lukp_a2_age1',
#  'lukp_a2_age2',
#  'lukp_a2_age3',
#  'lukp_a2_age4',
#  'lukp_a2_age5',
#  'lukcat_incomeband2',
#  'lukp_a2_income1',
#  'lukp_a2_income2',
#  'lukp_a2_income3',
#  'lukp_a2_income4',
#  'lukp_a2_income5',
#  'lukp_a2_income6a',
#  'lukp_a2_income6b',
#  'lukp_a2_income6c',
 'income_band',
#  'lukp_a2_hhd1',
#  'lukp_a2_hhd2',
#  'lukp_a2_hhd5',
#  'lukp_a2_hhd6',
 'lukcat_children',
 'lukcat_a2_childage',
 'lukcat_hhdsize',
 'lukcat_beds',
#  'lukp_occd2',
#  'lukp_occd3',
#  'lukp_occd5',
#  'lukp_occd6',
#  'lukp_occd8',
#  'lukp_occd9',
#  'lukp_occd10',
#  'lukp_occd11',
#  'lukp_occd12',
#  'lukp_a2_house5',
#  'lukp_a2_house6',
#  'lukp_a2_house7',
#  'lukp_a2_house8',
#  'house_type', 
 'house_type_revised',
           
#  'lukp_a2_house1',
#  'lukp_a2_house2',
#  'lukp_a2_house3',
#  'house_usage', 
 'house_usage_revised',
 'internet_frequent_user',
 'ips_intaddict',
 'ips_usetablet',
 'ips_useconsole',
 'lukp_mobhas',
 'ips_haspc',
 'oc14_bghtemailad',
 'oc14_willpaymore',
 'oc14_worthpaymore',
 'oc14_mobweb_wknd1',
 'oc14_mobweb_wknd2',
 'oc14_mobweb_wknd3',
 'ips_apps_games',
 'ips_apps_music',
 'ips_apps_transp',
 'ips_apps_date',
 'ips_apps_deliv',
 'ips_regaccessmobile2',
 'ips_regchat',
 'ips_tvfilmsonline',
 'ips_musicstreamfree',
 'ips_musicstreampaid',
 'ips_regpodcast',
 'ips_netflix',
 'ips_amazonprimetv',
 'oc14_vdemand_mob',
 'oc14_vdemand_pc',
 'oc14_vdemand_tv',
 'ips_vodwads',
 'oc14_buynewtech',
 'oc14_needmobnet',
 'oc14_cheapertech',
 'ips_acceptlbs',
 'ips_acceptvoa',
 'ips_followads',
 'ips_mobascheckout',
 'ips_mobentertain',
 'ips_mobimportant',
 'fr_smartband',
 'ips_smthomeappl',
 'ips_smthomeassist',
 'ips_smthomeutil',
 'ips_tvstreaming',
 'oc14_homesmartspeak',
 'oc14_homesmartlight',
 'oc14_homewifiplugs',
 'ips_wrlspeak',
 'ips_mobmanyapps',
 'ips_mobreggames',
 'ips_likesfactads',
 'oc14_bghttvad',
 'oc14_bghtnewspmagad',
 'oc14_bghtinsertad',
 'oc14_bghtnetad',
 'oc14_bghtmobilead',
 'oc14_bghtleafletad',
 'oc14_respdm',
 
 'has_eeros']

In [ ]:
model_df_cleaned=model_df[to_keep]

In [ ]:
cat_vars= model_df_cleaned.select_dtypes(include=['object'])
cat_vars
for var in cat_vars:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(model_df_cleaned[var], prefix=var)
    data1=model_df_cleaned.join(cat_list)
    model_df_cleaned=data1
    
data_vars=model_df_cleaned.columns.values.tolist()
to_keep=[i for i in data_vars if i not in cat_vars]

In [ ]:
model_df_cleaned[to_keep].shape

In [ ]:
# further selection of features to keep
eeros_cleaned_new = model_df_cleaned[to_keep]
eeros_cleaned_new= eeros_cleaned_new.drop(['broad_call_connections_broadband_tariff_Ultra','age_band_55-67', 'income_band_0-9999', 
                        'house_type_revised_Flat_or_maisonnette', 'house_usage_revised_private_renting'], axis=1)
eeros_cleaned_new= eeros_cleaned_new.set_index('customer_cust_account_number')
eeros_cleaned_new.head()

In [ ]:
eeros_cleaned_new['has_product'] = eeros_cleaned_new['has_eeros'] # this is done to add the target var at the end of dataframe
eeros_cleaned_new.drop(['has_eeros'], axis=1, inplace=True)

In [ ]:
eeros_cleaned_new.has_product.value_counts()

## PCA + Cosine

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

In [ ]:
pca=PCA(n_components=4)

In [ ]:
pca_eeros=pca.fit_transform(eeros_cleaned_new.iloc[:,:-1])

In [ ]:
pca.explained_variance_ratio_

In [ ]:
len(pca.components_)

In [ ]:
# prepare the pca_df for cosine later
pca_df=pd.DataFrame({'pca_1':pca_eeros[:,0],
             'pca_2':pca_eeros[:,1],
            'pca_3':pca_eeros[:,2],
                     'pca_4':pca_eeros[:,3],
             'has_product':eeros_cleaned_new.has_product})

In [ ]:
# Used the pca data to dervive eeros only customer and not eeros customers for cosine
eeros_customers = pca_df[pca_df.has_product==1]
non_eeros_customer = pca_df[pca_df.has_product==0]
non_eeros_customer.index.name = 'customer_cust_account_number_target'
print(eeros_customers.shape, non_eeros_customer.shape)

In [ ]:
cosine_mat = cosine_similarity(eeros_customers, non_eeros_customer)
len(cosine_mat)

In [ ]:
pca_cosine_df= pd.DataFrame(data=cosine_mat,    # values
             index=eeros_customers.index,    # 1st column as index
              columns = non_eeros_customer.index)

In [ ]:
pca_cosine_1=pca_cosine_df.stack().reset_index().copy()
pca_cosine_1.rename(columns={0: 'distance'},inplace=True)

In [ ]:
### lets select those customers which are most closest in terms of behaviour
threshhold = 0.935
pca_dup_result = pca_cosine_1[pca_cosine_1.distance > threshhold]

In [ ]:
# grpah to show the distribution of distance result and 90% threshold 
fig,ax=plt.subplots(figsize=(8,6))
sns.distplot(pca_cosine_1.distance,ax=ax)
ax.axvline(pca_cosine_1.distance.quantile(0.9),color='red',linestyle='--',linewidth=5)

In [ ]:
pca_dedup_result = pca_dup_result.drop_duplicates(subset=['customer_cust_account_number_target'])

In [ ]:
pca_customer_list=pca_dedup_result['customer_cust_account_number_target'].to_list()

In [ ]:
pca_test = eeros_cleaned_new[eeros_cleaned_new.index.isin(pca_customer_list)]
pca_test.shape

In [ ]:
pca_test.has_product.value_counts()

In [ ]:
# df for eeros only customers, not eeros customers
eeros_customers = eeros_cleaned_new[eeros_cleaned_new.has_product==1]
non_eeros_customer = eeros_cleaned_new[eeros_cleaned_new.has_product==0]
non_eeros_customer.index.name = 'customer_cust_account_number_target'
print(eeros_customers.shape, non_eeros_customer.shape)

In [ ]:
cosine_mat = cosine_similarity(eeros_customers, non_eeros_customer)
cosine_mat

In [ ]:
## convert cosine matrix to dataframe with rows as erros customer (ID) and columns as non eeros customers (ID) and values as their distance
cosine_df= pd.DataFrame(data=cosine_mat,    # values
             index=eeros_customers.index,    # 1st column as index
              columns = non_eeros_customer.index)

In [ ]:
### lets unstack to see the result as erros customer id, non eeros custoemr id and distance
cosine_df1 = cosine_df.stack().reset_index()
cosine_df1.head()

In [ ]:
cosine_df1.shape # this should be equal to dim(eeros customer data)*dim(non-eeros customer data)

In [ ]:
cosine_df2 = cosine_df1.rename(columns={0: 'distance'})

In [ ]:
### lets select those customers which are most closest in terms of behaviour
threshhold = 0.985
dup_result = cosine_df2[cosine_df2.distance > threshhold]

In [ ]:
### the above result can have duplicates, lets de duplicate them
dedup_result = dup_result.drop_duplicates(subset=['customer_cust_account_number_target'])

In [ ]:
customers_to_target = dedup_result['customer_cust_account_number_target'].to_list()

In [ ]:
### Check if the above list has all non erros customers
test = eeros_cleaned_new[eeros_cleaned_new.index.isin(customers_to_target)]
test.shape

In [ ]:
test.has_product.value_counts()

In [ ]:
# check the intersection of recommened customers' lists from Cosine only approach and PCA+Cosine approach
both_recommeded_lists=list(set(pca_customer_list).intersection(set(customers_to_target)))

In [ ]:
pd.DataFrame(pca_customer_list).to_csv('pca_eeros_list.csv')

In [ ]:
pd.DataFrame(both_recommeded_lists).to_csv('both_recommended_list_eeros.csv')

In [ ]:
len(pca_customer_list)

## Map

In [ ]:
distance_ranking = (unstacked_dis
                    .loc[unstacked_dis['from']
                         .isin(df.outcode.drop_duplicates())]
                    .join(partner_outcodes, on='to').dropna())

In [ ]:
_res=[]   ## update the dataframe names 
for r in distance_ranking.to_dict(orient='records'):
    _res+=[{'bb_outcode':r['from'],'partner_outcode':r['to'],
     'distance_km':r['distance_km'],'partner_position_id':p} for p in r['partner_position_id']]

In [ ]:
normalised_distances=pd.DataFrame(_res).sort_values(['bb_outcode','distance_km'])

In [ ]:
normalised_distances.to_csv('normalised_distances.csv')

In [ ]:
normalised_distances=pd.read_csv('normalised_distances.csv')
normalised_distances.drop(['Unnamed: 0'],axis=1,inplace=True)

In [ ]:
df.to_csv('df.csv')
bq_rfm.to_csv('bq_rfm.csv')

In [ ]:
normalised_distances=normalised_distances[normalised_distances.distance_km <= 10]
normalised_distances = normalised_distances.join(bq_rfm.set_index('partner_position_id'), 
                                                 on='partner_position_id', how='inner')
normalised_distances=(normalised_distances.loc[(normalised_distances.rfm_score >= 10)])

In [ ]:
normalised_distances=normalised_distances[['bb_outcode','partner_outcode','distance_km','partner_position_id','tenure_months','rfm_score','rfm_category','title_group']]

In [ ]:
bb_customer=df[['customer_cust_account_number','broad_call_connections_equipment','broad_call_connections_broadband_tariff',
    'has_eeros','outcode']]
filtered_data=normalised_distances.join(bb_customer.set_index('outcode'),on='bb_outcode', how='right')

In [ ]:
all_postcodes = pd.read_csv('ukpostcodes.csv')
all_postcodes['outcode'] = all_postcodes.postcode.str.split().str[0]
outcodes = outcodes.join(all_postcodes.dropna().groupby('outcode')['postcode'].count(), on='postcode', 
                         how='left', rsuffix='_count')
outcodes.dropna(inplace=True)

In [ ]:
bb_data=(filtered_data
 .drop_duplicates().join(outcodes.set_index('postcode'), 
       on='bb_outcode', how='inner')
.reset_index(drop=True).join(df.groupby('outcode').count()['customer_cust_account_number'],
                             on='bb_outcode', how='left',rsuffix='_count'))

In [ ]:
%pip install folium -qq

In [ ]:
partner_data = (filtered_data.loc[:, ['partner_position_id', 'rfm_score']]
 .join(partner_postcodes.set_index('partner_position_id')['postcode'], on='partner_position_id', how='left')
 .join(all_postcodes.set_index('postcode').loc[:, ['latitude', 'longitude']], on='postcode', how='left')
                .drop_duplicates()
)

In [ ]:
bb_data.dropna(inplace=True)
partner_data.dropna(inplace=True)

In [ ]:
non_eeros_pcode_heatmapdata = bb_customer.loc[bb_customer.has_eeros==0].join(all_postcodes.set_index('postcode').loc[:, ['latitude', 'longitude']], 
                    on='outcode', how='inner').loc[:, ['latitude', 'longitude']].values
eeros_live_pcode_hmdata = bb_customer.loc[bb_customer.has_eeros==1].join(all_postcodes.set_index('postcode').loc[:, ['latitude', 'longitude']], 
                    on='outcode', how='inner').loc[:, ['latitude', 'longitude']].values



In [ ]:
import folium
from folium.plugins import MarkerCluster 
from folium.plugins import HeatMap


In [ ]:
# Bounds
min_lat, max_lat = 48.77, 60
min_lon, max_lon = -9.05, 5

m = folium.Map(location=[53.884797 , -3.438721],
               tiles='Stamen Terrain', 
               zoom_start=6,
               min_lat=min_lat, 
               max_lat=max_lat,
               min_lon=min_lon, 
               max_lon=max_lon,
               max_zoom=18, 
               min_zoom=5)

bb_outcodes_feature_group = folium.FeatureGroup(name='Broadband outcodes').add_to(m) #Creates a Feature Group that adds its child layers into a parent group when added to a map (e.g. through LayerControl). 
                                                        # Useful to create nested groups, or cluster markers from multiple overlays
not_yet_eeros_feature_group = folium.FeatureGroup(name='Not Eeros postcodes').add_to(m)
eeros_live_feature_group = folium.FeatureGroup(name='Eeros live postcodes').add_to(m)

circle_colours = {'Low': 'red', 'Mid': 'yellow', 'High': 'green'}

for pc in bb_data.to_dict(orient='records'):
    popup = folium.Tooltip(f"""{pc['bb_outcode']}<br>Postcodes: {pc['bb_outcode']}""")
    marker = folium.Circle(location=(pc['latitude'], pc['longitude']), radius=5_000,
                           tooltip=popup, #(the popup message) 
                          ).add_to(m)
    marker.add_to(bb_outcodes_feature_group)
    
heatmap = HeatMap(non_eeros_pcode_heatmapdata, radius=25, control=False).add_to(m)
heatmap.add_to(not_yet_eeros_feature_group);

heatmap = HeatMap(eeros_live_pcode_hmdata, radius=25, control=False).add_to(m)
heatmap.add_to(eeros_live_feature_group);

In [ ]:
partners_feature_group = folium.FeatureGroup(name='Partners').add_to(m)

partner_cluster =  MarkerCluster(control=False).add_to(m)
partner_cluster.add_to(partners_feature_group)

for pc in partner_data.loc[partner_data.latitude.notna()].to_dict(orient='records'):
    popup = folium.Popup(f"{pc['partner_position_id']}<br>Postcode: {pc['postcode']}<br>RFM: {round(pc['rfm_score'], 2)}")
    icon = folium.map.Icon(color='darkpurple')
    marker = folium.Marker(location=(pc['latitude'], pc['longitude']), 
                           clustered_marker=True, popup=popup, icon=icon).add_to(partner_cluster)


In [ ]:
folium.LayerControl().add_to(m)
m